In this notebook, I will use pre-trained medium-sized GPT2 model from the [huggingface](https://github.com/huggingface/transformers) to generate some text.

In [ ]:
!pip install transformers

In [7]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

import torch
import numpy as np

from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

### Models and classes

I use [GPT2LMHeadModel](https://github.com/huggingface/transformers/blob/master/transformers/modeling_gpt2.py#L491) as the main language model, which is [GPT2Model](https://github.com/huggingface/transformers/blob/master/transformers/modeling_gpt2.py#L326), with an additional linear layer that uses input embedding layer weights to do the inverse operation of the embedding layer - the linear layer is required to create logits vector for the dictionary from outputs of the GPT2.

[GPT2Tokenizer](https://github.com/huggingface/transformers/blob/master/transformers/tokenization_gpt2.py#L106) is a byte-code pair encoder that will transform input text input into input tokens that the huggingface transformers were trained on. 

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [1]:
# Function to first select topN tokens from the probability list and then based on the selected N word distribution
# get random token ID
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

### Text generation

At each prediction step, GPT2 model needs to know all of the previous sequence elements to predict the next one. Below is a function that will tokenize the starting input text and then in a loop one new token is predicted at each step and is added to the input sequence. In the end, the token list is decoded back into a text. 

In [42]:
def generate_some_text(input_str, text_len = 250):

    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)

    model.eval()
    with torch.no_grad():

        for i in range(text_len):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=10) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(output_text)

In [35]:
generate_some_text(" The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. ")

The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It is the world of technology and information that is now being used to enslave you for a lifetime and to destroy you when you are young, as we know is going to happen. This technology, the Matrix, was created for this purpose. It is now being used by the Illuminati. And it's the world we live in that's being used to enslave us. You see, this world is not a dream. It is real, and if we want to know why, we need to look inside our own souls. It's called "love." It is the most powerful force in the world, and it's the only force that can save us from the Matrix. Love is the power that allows you to love someone in your life. Love is also your power of resistance. If

In [45]:
generate_some_text(" Artificial general intelligence is ")

Artificial general intelligence is the most likely future of the human race; it's a science which is not just possible but inevitable."

The AI industry, like most in technology, is based on a belief system of "the future is here". We've seen that with Google and Apple.

But this time round, we're not going to be able to see this future in the near future; we're going to have artificial intelligence as we know it today in a very short period of time. And that's not going to be good news for anyone in the industry, and certainly not the people who are making the AI.

The AI industry is currently in a very precarious place; it's very fragile, and very vulnerable to the kind of changes that are happening in the world today.

The biggest threat is going to be the rise of artificial intelligent technologies. AI is a very powerful concept, but it's not going to be able to completely replace the people who are making it, or the people that are using it, or the technology that it can use.

I t

In [59]:
generate_some_text(" The Godfather: \"I'm going to make him an offer he can't refuse.\" ")

The Godfather: "I'm going to make him an offer he can't refuse."

The Godfather: "What? What is it? He has to be a good boy? A good boy that doesn't want to be killed? Is the offer good?"

The Godfather: "He's a bad boy, isn't he."

The Godfather: "You're a good boy!"

The Godfather: "He's an idiot. He won't be able to understand what's going on!"

The Godfather: "You know, I never said you would be able to understand what's going on! I said you would be able to take him to a friend's house."

The Godfather: "I don't understand! You mean you'll never understand what's going on? What's happening to me?"

The Godfather: "That's the only way I can explain it to him. He's not going to be able to understand it either if I tell him what I know. He won't be able even to comprehend a thing if I tell him what it is."

The Godfather: "Well, you know, I've seen it all. I don't know what he will do. And, if he does, what's
